In [1]:
from data_sampler import WeightedSampler
from dataset import CARLADataset
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch
import os

In [3]:
path_ege_data = os.path.join("..", "data", "Dataset Ege")


# test commit
config = {"used_inputs": ["rgb", "depth", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle"],
        "seq_len": 2
        }

def transform(sample):
        """
        This function reshapes every input in a sample to be
        shape = [seq_len, data_shape]
        """
        for key in sample:
                data_tensor = torch.Tensor(sample[key])
                shape = list(data_tensor.shape)
                shape.remove(config["seq_len"])
                shape_new = [config["seq_len"]] + shape
                sample[key] = data_tensor.reshape(shape_new)
        return sample

# data_transform = transforms.Compose([
#         torch.stack()
#         # transforms.ToTensor(),
#         # torch.reshape(config["seq_len"], ...)
#         # transforms.Normalize(mean=[0.485, 0.456, 0.406],
#         #                      std=[0.229, 0.224, 0.225])
# ])

dataset = CARLADataset(root_dir=path_ege_data, config=config, transform=transform)
weighted_sampler = WeightedSampler(dataset=dataset)
print(dataset.__len__())

Town10HD_Scenario10_route9_11_28_17_59_39 has varying number of data files among input folders. It got discarded from the dataset.
2013


In [124]:
dl = DataLoader(dataset=dataset, batch_size=16, num_workers=0, sampler=weighted_sampler, collate_fn=None)

In [125]:
# Weird: When num_workers increases (>0) then it takes longer to loop over batches??
count = 0
for batch in dl:
    count += 1
print(count)

14


In [132]:
batch["depth"].shape

torch.Size([9, 1, 160, 960, 3])

In [127]:
batch["idx"]

tensor([[144],
        [204],
        [ 17],
        [102],
        [196],
        [ 20],
        [ 83],
        [134],
        [131]])

In [128]:
dataset.df_meta_data.iloc[190]

route           Town10HD_Scenario10_route3_11_28_17_28_17
depth                                            0126.png
rgb                                              0126.png
measurements                                    0126.json
Name: 126, dtype: object

In [133]:
batch

{'idx': tensor([[144],
         [204],
         [ 17],
         [102],
         [196],
         [ 20],
         [ 83],
         [134],
         [131]]),
 'depth': tensor([[[[[102., 187.,   4.],
            [ 73., 189.,   4.],
            [ 46., 191.,   4.],
            ...,
            [ 66., 186.,   2.],
            [ 41., 185.,   2.],
            [ 17., 184.,   2.]],
 
           [[204., 186.,   4.],
            [176., 188.,   4.],
            [148., 190.,   4.],
            ...,
            [ 66., 186.,   2.],
            [ 41., 185.,   2.],
            [ 17., 184.,   2.]],
 
           [[204., 186.,   4.],
            [175., 188.,   4.],
            [148., 190.,   4.],
            ...,
            [ 66., 186.,   2.],
            [ 41., 185.,   2.],
            [ 17., 184.,   2.]],
 
           ...,
 
           [[174.,  34.,   2.],
            [173.,  34.,   2.],
            [171.,  34.,   2.],
            ...,
            [246., 254.,   1.],
            [247., 254.,   1.],
       